---
# Évaluation des modèles de régression
---

<center><img src="https://python.gel.ulaval.ca/media/sio-u009/mlprocess_4.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans cette séquence nous allons évaluer, à l'aide des différentes métriques vues dans le cours, différents modèles de régression.

Dans un premier temps nous allons repartir de l'exemple simple vu dans le module d'introduction (notre polynôme de degré 5) et évaluer trois modèls différents. Dans un second temps nous allons évaluer le modèle de votre choix sur un ensemble de données public : Les données des habitants de Californie.

Importons d'abord les librairies nécessaires.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor

Générons les données sur notre polynôme de degré 5:

In [ ]:
X_lin = np.linspace(-10,10,100)[:,np.newaxis]
def generate_data(N):
    x = np.random.uniform(-10,10,N)
    y = np.polyval([0.03, 0.2, -1, -10, 100],x) + np.random.normal(0.0, 15.0, N)
    return x.reshape(-1, 1), y

In [ ]:
X,y = generate_data(200)

Et vérifions les données : 

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.plot(np.linspace(-10,10,100), np.polyval([0.03, 0.2, -1, -10, 100],np.linspace(-10,10,100)), color='black', linewidth=3)
ax.set_title('Data set')
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

## Préparation des données train/validation/test 40% / 30% / 30%

Découpage du dataset en trois partie:

In [ ]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.30)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.428)

## Calcul de l'erreur de Bayes sur validation

Voici un exemple de calcul des métriques $R^2$, erreur quadratique moyenne (MSE) et erreur absolue moyenne (MAE), si on avait le modèle parfait (i.e la fonction originale). 

In [ ]:
def f(x):
    return np.polyval([0.03, 0.2, -1, -10, 100],x)

R2 = r2_score(y_val, f(X_val))
MSE = mean_squared_error(y_val, f(X_val))
MAE = mean_absolute_error(y_val, f(X_val))
print('LR :', 'R2:', R2, 'MSE:', MSE, 'MAE:', MAE)

On ne peut donc pas faire mieux que ca sur ces métriques, cela nous donne une borne supérieure pour le $R^2$ et des bornes inférieures pour la MSE et la MAE.

## Évaluation des métriques de 3 modèles sur validation

En utilisant une régression linéaire ([`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)) voyons la performance que l'on peut obtenir : 

Essayez de visualiser le modèle résultat sur les données initiales pour valider visuelement le résultat (profitons en quand on peut).

In [ ]:
#...

En utilisant le noyau ci-dessous (dont vous pouvez adapter les hyperparamètres), reproduisez l'expérience avec un modèle de régression à base de processus Gaussien ([`GaussianProcessRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html))

In [ ]:
kernel = 100.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
               + WhiteKernel(noise_level=10, noise_level_bounds=(1e-10, 1e+3))

#...

Essayons enfin avec un perceptron multicouche ([`MLPRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html))

In [ ]:
# ... 

Essayez plusieurs hyperparamètres et trouver le meilleur modèle possible. Venez partager sur le forum le meilleur résultat que vous trouverez (en test ;-))

# Exemple 1 avec données réelles : [california housing](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

Essayons maintenant sur un ensemble de données réelles que vous avez probablement déja vu dans le module de préparation de données. 

C'est l'ensemble de données utilisé dans le deuxième chapitre du livre d'Aurélien Géron "Hands-On Machine learning with Scikit-Learn and TensorFlow". Il constitue une excellente introduction à la mise en œuvre d'algorithmes d'apprentissage machine car il nécessite un nettoyage rudimentaire des données, comporte une liste de variables facilement compréhensible et se situe à une taille optimale entre être trop ludique et trop encombrant.

Les données contiennent des informations provenant du recensement de 1990 en Californie. Ainsi, même si elles ne vous aident pas à prévoir les prix actuels du logement, comme d'autres ensemble de données plus récents, elles fournissent un ensemble de données d'introduction accessible pour enseigner aux gens les bases de l'apprentissage machine.

Récupérons les données et séparons les en trois sous ensembles (train-valid-test)



In [ ]:
from sklearn.datasets import fetch_california_housing
dataset = fetch_california_housing()
cal_X, cal_y = dataset.data, dataset.target

cal_X_train, cal_X_test, cal_y_train, cal_y_test = train_test_split(cal_X, cal_y, test_size=0.30, random_state=42)

### Entraînement et validation d'un régresseur sur california

Ici choisissons un regresseur (par exemple une régression [linéaire](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), un [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) ou un [MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html), mais vous pouvez choisir n'importe lequel [parmis tous ceux de SciKit-Learn](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)) et évaluons les en validation croisée (à l'aide de la fonction [cross_val_score()]()). 

M'hésitez pas à aller sur le forum pour discuter de vos meilleurs modèles et hyperparamètres !

In [ ]:
# ...

# Exemple 2 avec données réelles : [Diabète LARS](https://web.stanford.edu/~hastie/Papers/LARS/)

Dans cet ensemble de données, dix variables de base, l'âge, le sexe, l'indice de masse corporelle, la pression artérielle moyenne et six mesures de sérum sanguin ont été obtenues pour chacun des 44 patients diabétiques, ainsi que la réponse d'intérêt, une mesure quantitative de la progression de la maladie un an après la base.

In [ ]:
from sklearn.datasets import load_diabetes
dataset = load_diabetes()
dia_X, dia_y = dataset.data, dataset.target

dia_X_train, dia_X_test, dia_y_train, dia_y_test = train_test_split(dia_X, dia_y, test_size=0.30, random_state=42)

### Entraînement et validation d'un régresseur sur diabetes

Il s'agit de repratiquer encore l'exemple précédent avec un autre ensemble de données et donc un autre "meilleur" modèle.

M'hésitez pas à aller sur le forum pour discuter de vos meilleurs modèles et hyperparamètres !

In [ ]:
# ...